<a href="https://colab.research.google.com/github/DmitriySutyagin/Data-collection-and-markup/blob/main/Lesson_2_HTML_parsing_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
import pandas as pd
import json

In [2]:
name = []
price =[]
availability = []
descriptions = []
count = 0

In [3]:
ua = UserAgent()

headers = {"User-Agents": ua.random}

endpoint = 'http://books.toscrape.com/'

while True:

    count += 1
    print(f'Этап номер: {count}')

    respons = requests.get(endpoint, headers=headers)

    soup = BeautifulSoup(respons.content, 'html.parser')

    next_page = soup.find('li',('class', 'next'))
    result = soup.find_all('li', ('class', 'col-xs-6 col-sm-4 col-md-3 col-lg-3') )

    endpoint_2 = []


    for i in result:
        for link in i.find_all('div', ('class', 'image_container')):
            endpoint_2.append(link.find('a').get('href'))

    url_joined = []

    for link in endpoint_2:
        if endpoint != 'http://books.toscrape.com/':
            endpoint = 'http://books.toscrape.com/catalogue/'
            url_joined.append(endpoint +link)

        else:
            endpoint = 'http://books.toscrape.com/'
            url_joined.append(endpoint + link)

    for i in url_joined:

        response = requests.get(i)
        soup_2 = BeautifulSoup(response.content, 'html.parser')

        try:
            soup_name = soup_2.find('h1').text

            name.append(soup_name)

        except:
            print('Отсутствует название книги')

        try:
            soup_price = soup_2.find('p', ('class', 'price_color')).text
            soup_price = float(re.sub(r'[^\d.]', '', soup_price))
            price.append(soup_price)

        except:
            print('Цена товара отсутствует')
        try:
            soup_availability = soup_2.find('p', ('class', 'instock availability')).text
            soup_availability = int(re.sub(r'[^\d.]', '', soup_availability))
            availability.append(soup_availability)
        except:
            print('Отсутствуют данные о наличии')

        try:
            soup_descriptions = soup_2.find_all('p')
            for i in soup_descriptions:
               if len(i.get_text(strip=True)) >27:
                    descriptions.append(i.get_text(strip=True))
        except:
            print('Описание остутствует')

    output = {'Name' : name, 'Price': price, "Availabiliry" : availability, 'Descriptions' : descriptions}

    if  next_page == None:
            break

    if next_page == 'catalogue/page-2.html':

        endpoint = 'http://books.toscrape.com/'

        endpoint = endpoint + next_page

    else:
        endpoint = 'http://books.toscrape.com/catalogue/'
        *_, suffix = next_page.find('a').get('href').split('/')
        endpoint = endpoint + suffix


Этап номер: 1
Этап номер: 2
Этап номер: 3
Этап номер: 4
Этап номер: 5
Этап номер: 6
Этап номер: 7
Этап номер: 8
Этап номер: 9
Этап номер: 10
Этап номер: 11
Этап номер: 12
Этап номер: 13
Этап номер: 14
Этап номер: 15
Этап номер: 16
Этап номер: 17
Этап номер: 18
Этап номер: 19
Этап номер: 20
Этап номер: 21
Этап номер: 22
Этап номер: 23
Этап номер: 24
Этап номер: 25
Этап номер: 26
Этап номер: 27
Этап номер: 28
Этап номер: 29
Этап номер: 30
Этап номер: 31
Этап номер: 32
Этап номер: 33
Этап номер: 34
Этап номер: 35
Этап номер: 36
Этап номер: 37
Этап номер: 38
Этап номер: 39
Этап номер: 40
Этап номер: 41
Этап номер: 42
Этап номер: 43
Этап номер: 44
Этап номер: 45
Этап номер: 46
Этап номер: 47
Этап номер: 48
Этап номер: 49
Этап номер: 50


In [4]:
output = {'Name' : name, 'Price': price, "Availabiliry" : availability, 'Descriptions' : descriptions}
output
len(name), len(price), len(availability), len(descriptions)

(1000, 1000, 1000, 998)

In [5]:
with open('Lesson_2.json','w',encoding='utf-8') as f:
   json_file = json.dump(output, f, ensure_ascii=False,indent=2)

In [11]:
df = pd.DataFrame(name)
df

,0
0,A Light in the Attic
1,Tipping the Velvet
2,Soumission
3,Sharp Objects
4,Sapiens: A Brief History of Humankind
...,...
995,Alice in Wonderland (Alice's Adventures in Won...
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)"
997,A Spy's Devotion (The Regency Spies of London #1)
998,1st to Die (Women's Murder Club #1)


In [12]:
df = pd.DataFrame(price)
df

,0
0,51.77
1,53.74
2,50.10
3,47.82
4,54.23
...,...
995,55.53
996,57.06
997,16.97
998,53.98


In [13]:
df = pd.DataFrame(availability)
df

,0
0,22
1,20
2,20
3,20
4,20
...,...
995,1
996,1
997,1
998,1


In [14]:
df = pd.DataFrame(descriptions)
df

,0
0,It's hard to imagine a world without A Light i...
1,"""Erotic and absorbing...Written with starling ..."
2,"Dans une France assez proche de la nôtre, un h..."
3,"WICKED above her hipbone, GIRL across her hear..."
4,From a renowned historian comes a groundbreaki...
...,...
993,Friedrich Nietzsche's Beyond Good and Evil is ...
994,High school student Kei Nagai is struck dead i...
995,"In England’s Regency era, manners and elegance..."
996,"James Patterson, bestselling author of the Ale..."
